# Constraint Satisfaction Problems
As we saw, many AI problems can be modelled as a graph of states that a problem can be in. We then use a search algorithm to find a path in this graph that leads to the solution. One type of problems that can be solved in this manner are Constraint Satisfaction Problems. Constraints are basically conditions that cannot be violated during the process of solving the problem. When we arrive at the final solution, the states of the variables must obey all the constraints. 

Now, let us try to apply this to some real world problems. We will start with two relatively simple examples.

## 1. Hello World

We could try to implement the search algorithms ourselves (using our knowledge of graphs), but since this would be a little difficult, we will be using a library. Let us first use the python library *python-constraint* to solve a very basic constraint problem.

First install the library using following command:

In [1]:
pip install python-constraint

Note: you may need to restart the kernel to use updated packages.


In [2]:
# we will use the python range function

for i in range(5):
    print(i)

0
1
2
3
4


Now define a constraint problem with two variables within a specific range.

In [3]:
from constraint import *
problem = Problem()

problem.addVariable('a', range(8))
problem.addVariable('b', range(12))

Add the constraint the variables must obey.

In [4]:
# the lambda expression states that for every solution b must be two times a
problem.addConstraint(lambda a, b: a * 2 == b)

Let's see what solutions can be found. Check the results! Are the constraints satisfied?

In [5]:
solutions = problem.getSolutions()
print (solutions)

[{'a': 5, 'b': 10}, {'a': 4, 'b': 8}, {'a': 3, 'b': 6}, {'a': 2, 'b': 4}, {'a': 1, 'b': 2}, {'a': 0, 'b': 0}]


## 2. Numbers and values

One more simple example before getting to the real stuff. We will be using a package called *simpleai*. It contains various routines that are useful in building solutions using heuristic search techniques.

First install the library using following command:

In [6]:
pip install simpleai

Note: you may need to restart the kernel to use updated packages.


In [7]:
# we will use the python set function to remove duplicated values in a collection of values

print(set({ 1, 2, 3, 1, 2, 4, 3}))

{1, 2, 3, 4}


We will first import the necessary classes and define the problem.

In [8]:
from simpleai.search import CspProblem, backtrack

# we will try to find the value of four variables named number1, number2, number3, number4
variables = ('number1', 'number2', 'number3', 'number4')

# the list of values that each variable can take
#The values for each variable are chosen arbitrarily by the programmer to illustrate how the CSP solver works. 
# They are not based on any logic or rule.
domains = {
    'number1': [1, 2, 3],
    'number2': [1, 3],
    'number3': [2, 4],
    'number4': [2, 3, 4],
}

# define all the constraints, a constraint is a function with two parameters: variables and values
# the constraint returns true if the values obey the constraint, false otherwise

# a constraint that expects all the different variables to have different values
def constraint_unique(variables, values):
    # check if all the values are unique
    if len(values) == len(set(values)): # remove repeated values and count
        return True
    else:
        return False

# a constraint that expects the first variable to be bigger than the second
def constraint_bigger(variables, values):
    return values[0] > values[1] # short notation (if-then-else is not necessary)

# a constraint that expects two variables to be one odd and the other even
def constraint_odd_even(variables, values):
    if values[0] % 2 == 0:
        return values[1] % 2 == 1  # first even, expect second to be odd
    else:
        return values[1] % 2 == 0  # first odd, expect second to be even

Define the constraints for various scenarios. In this case, we specify three constraints as follows:
    
1. number1, number2 and number3 should be different values
2. number3 should be bigger than number2
3. if number1 is odd, then number4 value should be even and vice versa

And finaly search for a solution.

In [9]:
constraints = [
    (('number1', 'number2', 'number3'), constraint_unique),
    (('number3', 'number2'), constraint_bigger),
    (('number1', 'number4'), constraint_odd_even),
]

problem = CspProblem(variables, domains, constraints)

output = backtrack(problem)
print('\nSolutions:', output)


Solutions: {'number1': 1, 'number2': 3, 'number3': 4, 'number4': 2}


You can check the __domains__ and the __constraints__ to see if the solutions satisfy all those constraints. By the way, there is more then one solution, the search algorithm stops after finding the first one.

## 3. Sudoku - Exercise

Can you try to solve the following simplified sudoku puzzle? The aim is to fill the grid so that each row, column and box contains the same numbers (1 to 4).

<img src="./resources/sudoku.png"  style="height: 200px"/>

With a little more programming effort, you could use the same technique to solve a real sudoku (if you don't know what to do this evening).

In [10]:
from simpleai.search import CspProblem, backtrack



class SudokuCsp(CspProblem):

  """A Sudoku constraint satisfaction problem."""

  def __init__(self, puzzle):
    super().__init__(variables=range(len(puzzle) * len(puzzle[0])),
                     constraints=[self.row_constraint, self.col_constraint,
                               self.box_constraint])
    self.puzzle = puzzle

  def is_valid(self, assignment):
    """Checks if an assignment is valid for the Sudoku problem."""

    # Check if the assignment is complete.
    if len(assignment) != len(self.variables):
      return False

    # Check if the assignment satisfies all of the constraints.
    for constraint in self.constraints:
      if not constraint(assignment):
        return False

    return True

  def row_constraint(self, assignment):
    """Checks if the rows of the Sudoku puzzle are valid."""

    # Get the values of the current row.
    row = [assignment[i] for i in range(len(assignment)) if i // len(self.puzzle) == 0]

    # Check if all of the values in the row are unique.
    return len(set(row)) == len(row)

  def col_constraint(self, assignment):
    """Checks if the columns of the Sudoku puzzle are valid."""

    # Get the values of the current column.
    col = [assignment[i] for i in range(len(assignment)) if i % len(self.puzzle) == 0]

    # Check if all of the values in the column are unique.
    return len(set(col)) == len(col)

  def box_constraint(self, assignment):
    """Checks if the boxes of the Sudoku puzzle are valid."""

    # Get the values of the current box.
    box = [assignment[i] for i in range(len(assignment)) if i // len(self.puzzle) == 0 and i % len(self.puzzle) == 0]

    # Check if all of the values in the box are unique.
    return len(set(box)) == len(box)


# Load the Sudoku puzzle from the image.
puzzle = [[3, 0, 0, 0],
          [4, 0, 0, 0],
          [3, 0, 0, 0],
          [2, 0, 0, 0]]

# Create a Sudoku CSP problem.
sudoku_csp = SudokuCsp(puzzle)

# Solve the Sudoku puzzle using backtracking search.
solution = backtrack(sudoku_csp)

# Print the solved Sudoku puzzle.
print(solution)




TypeError: CspProblem.__init__() missing 1 required positional argument: 'domains'